In [9]:
from math import pi, sqrt

Given the solvent damping, what is the maximum rotation frequency we can expect from a given electric field? That is, at what frequency will the polarization and the dipole get out of sync?

A drag force coefficient

$ D = \frac{\sqrt{M k}}{Q} $

$ 2 \pi f_{res} = \sqrt{k/M} $

$ k = (2 \pi f_{res})^2 M$

$ D = \frac{\sqrt{M^2 (2 \pi f_{res})^2 }}{Q} $

assuming that the drag force is exclusively applied to the outer envelope,

$ F_{tangential} = D v $

$ v = d_{virus} \pi f_{rot} $

$ \tau_{friction} = D \pi f_{rot}(d_{virus}^2)/2  $

$ p = q \cdot d_q $

$ \tau_{dipole} = p E $

$ p E = D \pi f_{rot} (d_{virus}^2)/2 $ 

$ f_{rot} = \frac{p E}{D (\pi) (d_{virus}^2)/2} $ 


where

    p = dipole moment
    q = separated charge (1/2 total charge), C
    M = mass in mass-spring system 
    k = spring constant of mass-spring system
    Q = virus quality factor
    d_virus = virus envelope diameter, meters
    E = electric field, V/m
    f_res = resonant frequency of linear, dipole mode, Hz
    f_rot = frequency of rotational mode, Hz
    v = tangential velocity at envelope surface
    


In [24]:
E = 50.0 # V/m
e = 1.602e-19
q = 3.0*1e7*e #C

d_virus = 120e-9
d_q = 10e-9 

M = 1.4e-19 #80 MDa - probably wrong

Q = 2.0
f_res = 8e9

p = q*d_q

D = sqrt((M**2.0)*((2.0*pi*f_res)**2.0))/Q

f_rot = (p*E)/(D*pi*((d_virus**2.0)/2.0))

f_rot

30192.808071991283

So, assuming that the translational and rotational friction coefficients are equal, the equilibrium resonant frequency is 1000x too small to rupture the virus.

However, the former is not a good assumption. Unlike in translational vibration, where the solvent must be inertially displaced, according to [Steele 1963] a perfect sphere can undergo perfect slip with the surrounding solvent.

(This also means that for molecular dynamics simulation, implicit solvation techniques using Langevin integration with a fixed damping coefficient is not a good match for this problem).

Via [Hu 1974], a small degree of oblateness or prolateness can affect the rotational friction coefficient decisively. But if the virus tends to rotate along its minimum friction axis, this effect should not be pronounced.

The virus's surface is also not smooth; spike proteins and other clutter may also slow its rotation.

In [23]:
I = 2.0/5.0 * M * ((0.5*d_virus)**2.0)


tau = p*E #torque

x = tau/I # 10
x

1.191964285714286e+16